In [ ]:
# ruff: noqa: F401

In [ ]:
%load_ext autoreload
%load_ext pyinstrument

%autoreload 2

In [ ]:
import sys

sys.path.insert(0, "..")

In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
import json
from datetime import datetime, timedelta
from pathlib import Path

import hvplot.polars
import numpy as np
import polars as pl

In [ ]:
from olap_benchmarks.dbs.monetdb import MonetDB
from olap_benchmarks.dbs.monetdb.fetch import fetch_binary, fetch_pymonetdb
from olap_benchmarks.dbs.monetdb.insert import insert
from olap_benchmarks.dbs.utils import drop_table

db = MonetDB()

In [ ]:
df = pl.read_parquet("../data/input/time_series/data_0.2M_0.5k.parquet").with_columns(
    (100 * pl.col.col_1 - 50).cast(pl.Int8).alias("col_1_int8"),
    (100 * pl.col.col_1 - 50).cast(pl.Int16).alias("col_1_int16"),
    (100 * pl.col.col_1 - 50).cast(pl.Int32).alias("col_1_int32"),
    (100 * pl.col.col_1 - 50).cast(pl.Int64).alias("col_1_int64"),
    pl.col.col_1.cast(pl.Float32).alias("col_1_float32"),
    pl.col.col_1.cast(pl.Float64).alias("col_1_float64"),
    pl.col.col_1.cast(pl.Decimal(18, 3)).alias("col_1_decimal_default"),
    pl.col.col_1.cast(pl.Decimal(14, 5)).alias("col_1_decimal_14_5"),
    pl.col.col_1.cast(pl.Decimal(10, 2)).alias("col_1_decimal_10_2"),
    pl.col.col_1.cast(pl.String).alias("col_1_str"),
    (pl.col.col_1 > 0.5).cast(pl.Boolean).alias("col_1_bool"),
    pl.col.col_1.cast(pl.Binary).alias("col_1_blob"),
    ('{ "val": ' + pl.col.col_1.cast(pl.String) + "}").alias("col_1_json_string"),
    ('{ "val": ' + pl.col.col_1.cast(pl.String) + "}").map_elements(json.loads, pl.Object).alias("col_1_json_obj"),
    ('{ "val": ' + pl.col.col_1.cast(pl.String) + "}")
    .str.json_decode(infer_schema_length=100)
    .alias("col_1_json_struct"),
    pl.col.time.cast(pl.Time).alias("time_time"),
    pl.col.time.cast(pl.Date).alias("time_date"),
)


df = df.with_columns(
    pl.when((pl.col.time > df.get_column("time")[4]) | (pl.col.time == df.get_column("time")[0]))
    .then(pl.col(n))
    .otherwise(None)
    for n in df.columns
    if n != "time"
)

# df = df.select([n for n in df.columns if n == "time" or n.count("_") == 2])

df

In [ ]:
TABLE_NAME = "test_insert"

In [ ]:
drop_table(TABLE_NAME, db.connect())

In [ ]:
insert(df, TABLE_NAME, db.connect(), primary_key="time")

In [ ]:
fetch_binary(f"select * from {TABLE_NAME} order by time", db.connect())

In [ ]:
%%pyinstrument

drop_table(TABLE_NAME, db.connect())
insert(df, TABLE_NAME, db.connect(), primary_key="time")

* JSON is read differently by binary and pymonetdb due to a bug in pymonetdb (ignore `col_1_json`)


In [ ]:
from utils import compare

In [ ]:
drop = [n for n in df.columns if "json" in n]

In [ ]:
df_pymonetdb = fetch_pymonetdb(f"select * from {TABLE_NAME} order by time", db.connect()).drop(drop, strict=False)

In [ ]:
df_binary = fetch_binary(f"select * from {TABLE_NAME} order by time", db.connect()).drop(drop, strict=False)

In [ ]:
compare(df_pymonetdb, df_binary)

In [ ]:
compare(df_pymonetdb, df.drop(drop, strict=False).sort("time"))

In [ ]:
compare(df_binary, df.drop(drop, strict=False).sort("time"))

In [ ]:
df